In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("Lab7").getOrCreate()

24/11/03 21:15:00 WARN Utils: Your hostname, ankurPC resolves to a loopback address: 127.0.1.1; using 10.87.1.9 instead (on interface wlp0s20f3)
24/11/03 21:15:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 21:15:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("stock_data.csv", header=True, inferSchema=True)

In [4]:
returns = df.select("Close").rdd.flatMap(lambda x: x).collect()
returns = np.diff(returns) / returns[:-1]    

In [5]:
drift = np.mean(returns)
volatility = np.std(returns)

In [6]:
def monte_carlo(start, drift, volatility, num=252, sims=1000):
    results = []
    for _ in range(sims):
        prices = [start]
        for _ in range(num):
            shock = np.random.normal(0, 1)  
            price = prices[-1] * np.exp(drift + volatility * shock)
            prices.append(price)
        results.append(prices)
    return results

In [7]:
start = df.select("Close").tail(1)[0][0]  
steps = 252 
sims = 1000  

In [8]:
results = spark.sparkContext.parallelize([start] * sims).map(
    lambda start: monte_carlo(start, drift, volatility, steps, 1)[0]
)

In [9]:
simulated_prices = results.collect()
simulated_prices_df = pd.DataFrame(simulated_prices).T
# simulated_prices_df.columns = [f'Simulation_{i+1}' for i in range(sims)]
print(simulated_prices_df.head())

         0          1          2          3          4          5    \
0  52.950000  52.950000  52.950000  52.950000  52.950000  52.950000   
1  53.070409  53.070409  53.070409  53.070409  53.070409  53.070409   
2  53.191092  53.191092  53.191092  53.191092  53.191092  53.191092   
3  53.312050  53.312050  53.312050  53.312050  53.312050  53.312050   
4  53.433282  53.433282  53.433282  53.433282  53.433282  53.433282   

         6          7          8          9    ...        990        991  \
0  52.950000  52.950000  52.950000  52.950000  ...  52.950000  52.950000   
1  53.070409  53.070409  53.070409  53.070409  ...  53.070409  53.070409   
2  53.191092  53.191092  53.191092  53.191092  ...  53.191092  53.191092   
3  53.312050  53.312050  53.312050  53.312050  ...  53.312050  53.312050   
4  53.433282  53.433282  53.433282  53.433282  ...  53.433282  53.433282   

         992        993        994        995        996        997  \
0  52.950000  52.950000  52.950000  52.950000

In [10]:
spark.stop()